In [1]:
#This notebook builds a similarity merger from baseline randomforest 
#Morgan fingerprint 
#and cell painting

In [2]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint

In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [4]:
similarity = pd.read_csv("Similarity_def_mean_train_test_no_Class.csv")
similarity

,assay,InChICode_standardised,ts,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives,Data
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.446814,0.000000,0.396191,0.000000,0.433559,0.000000,0.442869,0.000000,0.416542,0.000000,0.430447,0.000000,Training
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.443323,0.000000,0.378563,0.000000,0.414811,0.000000,0.689046,0.139860,0.662077,0.222222,0.678060,0.214953,Training
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.440373,0.000000,0.373476,0.000000,0.428608,0.000000,0.729628,0.143357,0.676574,0.152778,0.719979,0.186916,Training
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.478412,0.006993,0.332100,0.000000,0.478412,0.009346,0.536519,0.000000,0.458671,0.013889,0.526597,0.018692,Training
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.473341,0.000000,0.401539,0.000000,0.420060,0.000000,0.707402,0.055944,0.599794,0.055556,0.673625,0.074766,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.498732,0.006969,0.479915,0.022222,0.314529,0.000000,0.645185,0.045296,0.605005,0.111111,0.628868,0.055838,HeldOut
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.436372,0.000000,0.394652,0.000000,0.423370,0.000000,0.584091,0.020906,0.532951,0.044444,0.584091,0.060914,HeldOut
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.481728,0.003484,0.331436,0.000000,0.471541,0.005076,0.725631,0.149826,0.685987,0.244444,0.723294,0.167513,HeldOut
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.380949,0.000000,0.279776,0.000000,0.380949,0.000000,0.812100,0.118467,0.697407,0.100000,0.812100,0.142132,HeldOut


In [5]:
df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv")
df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,True,False,0.158783,0.500219,0.0,True
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,False,False,0.508641,0.530402,1.0,False
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,True,True,0.262399,0.319723,0.0,True
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,False,True,0.100980,0.628169,0.0,False
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,False,True,0.527003,0.289754,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False,0.727044,0.698855,1.0,False
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True,0.214095,0.398324,0.0,True
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True,0.221627,0.651916,0.0,False
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True,0.163395,0.450736,0.0,True


In [6]:
list_of_lists_df = pd.merge(df, similarity, on=["assay","InChICode_standardised","Data"])
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,...,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,...,0.396191,0.000000,0.433559,0.000000,0.442869,0.000000,0.416542,0.000000,0.430447,0.000000
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,...,0.378563,0.000000,0.414811,0.000000,0.689046,0.139860,0.662077,0.222222,0.678060,0.214953
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,...,0.373476,0.000000,0.428608,0.000000,0.729628,0.143357,0.676574,0.152778,0.719979,0.186916
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,...,0.332100,0.000000,0.478412,0.009346,0.536519,0.000000,0.458671,0.013889,0.526597,0.018692
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,...,0.401539,0.000000,0.420060,0.000000,0.707402,0.055944,0.599794,0.055556,0.673625,0.074766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,...,0.479915,0.022222,0.314529,0.000000,0.645185,0.045296,0.605005,0.111111,0.628868,0.055838
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,...,0.394652,0.000000,0.423370,0.000000,0.584091,0.020906,0.532951,0.044444,0.584091,0.060914
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,...,0.331436,0.000000,0.471541,0.005076,0.725631,0.149826,0.685987,0.244444,0.723294,0.167513
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,...,0.279776,0.000000,0.380949,0.000000,0.812100,0.118467,0.697407,0.100000,0.812100,0.142132


In [7]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
from itertools import product
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from collections import Counter
from numpy import argmax

In [8]:
def RF_Fn(assay):

    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    
    
    proba_batch=[]
    pred_batch=[]
    true_batch=[]
    StdInChI_batch=[]
    
    train= detail_list[detail_list["Data"]=="Training"]
    #test
    test= detail_list[detail_list["Data"]=="HeldOut"]
    
    #columns = ["ts", "ts_perc", "ts_actives", "ts_perc_actives", 
    #          "ts_inactives", "ts_perc_inactives", "pc", "pc_perc", "pc_actives", "pc_perc_actives", 
    #          "pc_inactives", "pc_perc_inactives"]
    
    columns = ["ts_actives", "pc_actives", "fp_proba_scaled", "CP_proba_scaled"]
    
    
    print("Model for training data")    

    X_train = train[columns].to_numpy()
    y_train = train["true"].to_numpy()
    X_test = test[columns].to_numpy()
    
    print(len(X_train))
    print(len(X_test))
    
    #print(X_train)
    
    classifier = LogisticRegression(random_state=42, class_weight="balanced")
    classifier.fit(X_train, y_train)
        
    StdInChI = test["InChICode_standardised"].to_numpy()
    pred = classifier.predict(X_test) 
    proba = classifier.predict_proba(X_test)[:,1]
    true = test["true"].to_numpy()
            

    StdInChI_batch = np.concatenate([StdInChI_batch, StdInChI])
    proba_batch = np.concatenate([proba_batch, proba])
    pred_batch = np.concatenate([pred_batch, pred])
    true_batch = np.concatenate([true_batch, true])

    print(len(pred_batch))
    print(len(true_batch))
   
    return StdInChI_batch, true_batch, pred_batch, proba_batch

In [9]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import KFold

def check_heirarchial(assay, method, y_true, y_pred, y_prob):
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    #print(conf_matrix)
    
    #print(classification_report(y_true, y_pred))
    
    ba= balanced_accuracy_score(y_true, y_pred)
    #print('balanced_accuracy_score ',ba)
    
    mcc=matthews_corrcoef(y_true, y_pred)
    #print('matthews_corrcoef ',mcc)
    # Sensitivity, hit rate, recall, or true positive rate
    
    f1= f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    
    #print('F1 Toxic', f1)
    #print('precision', precision)
    
    Specificity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    # Specificity or true negative rate
    Sensitivity = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1]) 
    
    #print( 'Sensitivity', Sensitivity)
    #print( 'Specificity', Specificity)
    AUC = roc_auc_score(y_true, y_prob)
    #print('AUC-ROC ',AUC)    
    # calculate roc curves
    best_model_fpr, best_model_tpr, _ = roc_curve(y_true, y_prob)   
    
    
    row=[assay, method, f1, precision, Sensitivity, Specificity, ba, mcc, AUC]
    metrics_list.append(row)
    
    return metrics_list

In [10]:
import warnings
warnings.filterwarnings("ignore") #ignore, default
import pandas

#sns.set(rc={'figure.figsize':(2, 2)})
#plt.rcParams['figure.dpi'] = 80
#sns.set_style("white")
metrics_list=[]
all_preds_test_compounds=pd.DataFrame(columns=['assay', 'InChICode_standardised', 'fp_proba', 'fp_pred', 'fp_threshold', 'CP_proba',
       'CP_pred', 'CP_threshold', 'true', 'ts', 'pc', 'Data', 'MFP_Correct',
       'CP_Correct', 'fp_proba_scaled', 'CP_proba_scaled', 'simple_pred',
       'Ensemble_Correct', 'merger_pred', 'merger_Correct'])

#for assay in tqdm(assaylist[:7]):
for assay in tqdm(list_of_lists_df.assay.unique()[:]):
    
    print("Assay: ", assay)
    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    detail_list_test = detail_list[detail_list["Data"]=="HeldOut"]

    #print("CP")
    check_heirarchial(assay, "CP", detail_list_test.true, detail_list_test.CP_pred, detail_list_test.CP_proba)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="CP_Correct", legend=False)
    #plt.show()
    
    #print("MFP")
    check_heirarchial(assay, "MFP", detail_list_test.true, detail_list_test.fp_pred, detail_list_test.fp_proba)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="MFP_Correct", legend=False)
    #plt.show()
    
    #print("Ensemble Averaged")
    check_heirarchial(assay, "Ensemble Averaged", detail_list_test.true, detail_list_test.simple_pred, detail_list_test.simple_pred)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="simple_pred", legend=False)
    #plt.show()
    
    #print("Distance Based Merger")
    merger_df=pd.DataFrame()
    
    StdInChI_batch, true_batch, pred_batch, proba_batch = RF_Fn(assay)
    check_heirarchial(assay, "Distance Based Merger", true_batch, pred_batch, proba_batch)
    
    #print(true_batch)
    
    print(assay)
    
    merger_df["InChICode_standardised"]=StdInChI_batch
    merger_df["merger_pred"]= pred_batch
    merger_df["true"]=true_batch
    merger_df["merger_Correct"]= merger_df["merger_pred"] == merger_df["true"]
    merger_df["assay"]=assay
    
    #.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="herrachial_Correct", legend=False)
    
    #For each assay combine original predictions and merger_df predictions
    #all_preds = pd.DataFrame()
    all_preds_test_compounds= pd.concat([all_preds_test_compounds, pd.merge(detail_list_test, merger_df)])

  3%|█▌                                          | 3/88 [00:00<00:03, 26.18it/s]

Assay:  588458
Model for training data
358
90
90
90
588458
Assay:  588334
Model for training data
422
106
106
106
588334
Assay:  2642
Model for training data
611
153
153
153
2642
Assay:  2156
Model for training data
454
114
114
114
2156
Assay:  2330
Model for training data
262
66
66
66
2330
Assay:  2216
Model for training data
473
119


 10%|████▌                                       | 9/88 [00:00<00:02, 27.60it/s]

119
119
2216
Assay:  743015
Model for training data
282
71
71
71
743015
Assay:  504444
Model for training data
860
216
216
216
504444
Assay:  894
Model for training data
924
232
232
232
894
Assay:  720635
Model for training data
204
51
51
51
720635
Assay:  1688
Model for training data
486
122
122
122
1688
Assay:  2599


 17%|███████▎                                   | 15/88 [00:00<00:02, 27.51it/s]

...

66
66
651820
Assay:  624297
Model for training data
616
155
155
155
624297
Assay:  651635
Model for training data
283
71
71
71
651635
Assay:  624417
Model for training data
676
170
170
170
624417
Assay:  624202
Model for training data
303
76
76
76
624202
Assay:  624287
Model for training data
83
21
21
21
624287
Assay:  624288


 92%|███████████████████████████████████████▌   | 81/88 [00:02<00:00, 28.05it/s]

Model for training data
124
31
31
31
624288
Assay:  651644
Model for training data
121
31
31
31
651644
Assay:  652104
Model for training data
315
79
79
79
652104
Assay:  720579
Model for training data
84
22
22
22
720579
Assay:  720533
Model for training data
144
36
36
36
720533
Assay:  720542
Model for training data
83
21
21
21
720542
Assay:  720580


100%|███████████████████████████████████████████| 88/88 [00:03<00:00, 27.82it/s]

Model for training data
96
25
25
25
720580
Assay:  720504
Model for training data
102
26
26
26
720504
Assay:  720532
Model for training data
240
61
61
61
720532
Assay:  1159524
Model for training data
704
176
176
176
1159524
Assay:  1117304
Model for training data
308
78
78
78
1117304
Assay:  1117305
Model for training data
287
72
72
72
1117305


In [11]:
all_preds_test_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,...,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives
0,588458,InChI=1S/C14H15BrN2O3/c1-17(2)6-5-16-13(18)11-...,0.335765,1,0.310223,0.478519,1,0.304308,0.0,0.397291,...,0.000000,0.334683,0.000000,0.380251,0.000000,0.000000,0.311134,0.000000,0.325709,0.000000
1,588458,InChI=1S/C23H22N2O2/c26-23(16-22-21-9-5-4-6-17...,0.210945,0,0.310223,0.168993,0,0.304308,0.0,0.482684,...,0.000000,0.383006,0.000000,0.482684,0.000000,0.097765,0.627549,0.088889,0.698505,0.164179
2,588458,InChI=1S/C22H17F2NO3/c23-18-7-1-15(2-8-18)13-2...,0.155810,0,0.310223,0.248408,0,0.304308,0.0,0.405553,...,0.000000,0.307502,0.000000,0.398721,0.000000,0.016760,0.420571,0.011111,0.751601,0.022388
3,588458,InChI=1S/C24H24N6O4/c25-22-20(24(31)26-7-8-29-...,0.163149,0,0.310223,0.208577,0,0.304308,0.0,0.417348,...,0.000000,0.364818,0.000000,0.412587,0.000000,0.064246,0.585947,0.100000,0.630570,0.100746
4,588458,InChI=1S/C15H23BrN2O4S/c1-10-8-18(11(2)9-19)23...,0.285539,0,0.310223,0.554642,1,0.304308,0.0,0.451498,...,0.002793,0.329565,0.000000,0.451498,0.003731,0.053073,0.772584,0.155556,0.756990,0.063433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,0.498732,...,0.006969,0.479915,0.022222,0.314529,0.000000,0.045296,0.605005,0.111111,0.628868,0.055838
68,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,0.436372,...,0.000000,0.394652,0.000000,0.423370,0.000000,0.020906,0.532951,0.044444,0.584091,0.060914
69,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,0.481728,...,0.003484,0.331436,0.000000,0.471541,0.005076,0.149826,0.685987,0.244444,0.723294,0.167513
70,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,0.380949,...,0.000000,0.279776,0.000000,0.380949,0.000000,0.118467,0.697407,0.100000,0.812100,0.142132


In [12]:
all_preds_test_compounds.to_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble_merger_test_compounds.csv", index=False)

In [13]:
metrics_list = pd.DataFrame(metrics_list,columns=["assay","method","f1", "precision", "Sensitivity", "Specificity", "ba", "mcc", "AUC"])
metrics_list = metrics_list.sort_values(by=["assay", "method"]).reset_index(drop=True)
metrics_list

,assay,method,f1,precision,Sensitivity,Specificity,ba,mcc,AUC
0,1117304,CP,0.350000,0.466667,0.280000,0.849057,0.564528,0.152817,0.555472
1,1117304,Distance Based Merger,0.360000,0.360000,0.360000,0.698113,0.529057,0.058113,0.541132
2,1117304,Ensemble Averaged,0.000000,0.000000,0.000000,0.962264,0.481132,-0.111414,0.481132
3,1117304,MFP,0.235294,0.444444,0.160000,0.905660,0.532830,0.095910,0.520755
4,1117305,CP,0.461538,0.321429,0.818182,0.240000,0.529091,0.064466,0.473636
...,...,...,...,...,...,...,...,...,...
347,Novartis1,MFP,0.222222,1.000000,0.125000,1.000000,0.562500,0.297560,0.595588
348,Novartis2,CP,0.125000,0.142857,0.111111,0.647059,0.379085,-0.259376,0.431373
349,Novartis2,Distance Based Merger,0.307692,0.500000,0.222222,0.882353,0.552288,0.137890,0.575163
350,Novartis2,Ensemble Averaged,0.000000,0.000000,0.000000,0.941176,0.470588,-0.145521,0.470588


In [14]:
metrics_list.to_csv("All_metrics.csv", index=False)